In [1]:
# Main import block
import asyncio
import datetime
import signal
import time
from io import StringIO
import warnings
from contextlib import contextmanager
from datetime import date, datetime

# Sneaky scrapper
import cfscrape

# Upload to google sheets
import gspread
import numpy as np
import pandas as pd
import requests

# Upload to yandex
import yadisk
from bs4 import BeautifulSoup

# import df2gspread as d2g
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


warnings.filterwarnings("ignore")

In [6]:
url = "https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.\
Posted=True&UniDbQuery.so=1&UniDbQuery.mode=1&UniDbQuery.\
date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ=R01235&UniDbQuery.\
From=01.01.2023&UniDbQuery.To=09.04.2024"

In [7]:
scraper = cfscrape.create_scraper()
scraped_data = scraper.get(url)

In [76]:
# Получаем датафрейм
data = pd.read_html(scraped_data.text.replace(",", "."))[0]
data.columns = data.iloc[1]

# Преобразуем
data = data.iloc[2:]
data = data.rename(
    columns={data.columns[0]: "date", data.columns[1]: "unit", data.columns[2]: "nominal"}
).reset_index(drop=True)

# Cкидываем название индекса
data = pd.read_csv(StringIO(data.to_string(index=False)), delim_whitespace=True)
data["date"] = pd.to_datetime(data["date"], format='%d.%m.%Y')
data["unit"] = pd.to_numeric(data["unit"])
data["nominal"] = pd.to_numeric(data["nominal"])

data = data.sort_values("date").reset_index(drop=True)
data

,date,unit,nominal
0,2023-01-10,1,70.3002
1,2023-01-11,1,69.6094
2,2023-01-12,1,69.0202
3,2023-01-13,1,67.7775
4,2023-01-14,1,67.5744
...,...,...,...
305,2024-04-03,1,92.5254
306,2024-04-04,1,92.3892
307,2024-04-05,1,92.3058
308,2024-04-06,1,92.4155


In [4]:
async def cb_currency():
    async def get_currency(currency):
        current_date = datetime.today().strftime("%d.%m.%Y")

        dict_of_currencies = {
            "USD": "R01235",
            "EUR": "R01239",
            "Australian_Dollar": "R01010",
            "China_Yuan": "R01375",
            "British_Pound": "R01035",
            "Kazakhstan_Tenge": "R01335",
            "Japanese_Yen": "R01820",
            "Swiss_Franc": "R01775",
        }

        url = f"https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.\
Posted=True&UniDbQuery.so=1&UniDbQuery.mode=1&UniDbQuery.\
date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ={dict_of_currencies[currency]}&UniDbQuery.\
From=01.01.2023&UniDbQuery.To={current_date}"

        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)
        # time.sleep(2)
        
        return scraped_data

    def data_reconstruction(scraped_data, name):

        # Получаем датафрейм
        data = pd.read_html(scraped_data.text.replace(",", "."))[0]
        data.columns = data.iloc[1]

        # Преобразуем
        data = data.iloc[2:]
        data = data.rename(
            columns={
                data.columns[0]: "date",
                data.columns[1]: "unit",
                data.columns[2]: "nominal",
            }
        ).reset_index(drop=True)

        # Cкидываем название индекса
        data = pd.read_csv(StringIO(data.to_string(index=False)), delim_whitespace=True)
        data["date"] = pd.to_datetime(data["date"], format="%d.%m.%Y")
        data["unit"] = pd.to_numeric(data["unit"])
        data["nominal"] = pd.to_numeric(data["nominal"])

        data = data.sort_values("date").reset_index(drop=True)

        print(f"{name} is done!")
        display(data.tail())

        return data

    raw_usd = await get_currency(currency="USD")
    raw_eur = await get_currency(currency="EUR")
    raw_ausd = await get_currency(currency="Australian_Dollar")
    raw_yuan = await get_currency(currency="China_Yuan")
    raw_pound = await get_currency(currency="British_Pound")
    raw_tenge = await get_currency(currency="Kazakhstan_Tenge")
    raw_yen = await get_currency(currency="Japanese_Yen")
    raw_swiss = await get_currency(currency="Swiss_Franc")

    usd = data_reconstruction(raw_usd, name="USD")
    eur = data_reconstruction(raw_eur, name="EUR")
    ausd = data_reconstruction(raw_ausd, name="Australian_Dollar")
    yuan = data_reconstruction(raw_yuan, name="China_Yuan")
    pound = data_reconstruction(raw_pound, name="British_Pound")
    tenge = data_reconstruction(raw_tenge, name="Kazakhstan_Tenge")
    yen = data_reconstruction(raw_yen, name="Japanese_Yen")
    swiss = data_reconstruction(raw_swiss, name="Swiss_Franc")

    with pd.ExcelWriter(
        "./data/cb_currency.xlsx",
        date_format="YYYY-MM-DD",
        datetime_format="YYYY-MM-DD",
    ) as writer:
        usd.to_excel(writer, sheet_name="USD")
        eur.to_excel(writer, sheet_name="EUR")
        ausd.to_excel(writer, sheet_name="Australian_Dollar")
        yuan.to_excel(writer, sheet_name="China_Yuan")
        pound.to_excel(writer, sheet_name="British_Pound")
        tenge.to_excel(writer, sheet_name="Kazakhstan_Tenge")
        yen.to_excel(writer, sheet_name="Japanese_Yen")
        swiss.to_excel(writer, sheet_name="Swiss_Franc")
        

    return None

In [5]:
await cb_currency()

USD is done!


,date,unit,nominal
305,2024-04-03,1,92.5254
306,2024-04-04,1,92.3892
307,2024-04-05,1,92.3058
308,2024-04-06,1,92.4155
309,2024-04-09,1,92.5810


EUR is done!


,date,unit,nominal
305,2024-04-03,1,99.4116
306,2024-04-04,1,99.4277
307,2024-04-05,1,100.0668
308,2024-04-06,1,100.1259
309,2024-04-09,1,100.2236


Australian_Dollar is done!


...

,date,unit,nominal
305,2024-04-03,100,20.7196
306,2024-04-04,100,20.6442
307,2024-04-05,100,20.6542
308,2024-04-06,100,20.6982
309,2024-04-09,100,20.7218


Japanese_Yen is done!


,date,unit,nominal
305,2024-04-03,100,60.9924
306,2024-04-04,100,60.9749
307,2024-04-05,100,60.8797
308,2024-04-06,100,61.2348
309,2024-04-09,100,61.0009


Swiss_Franc is done!


,date,unit,nominal
305,2024-04-03,1,101.8329
306,2024-04-04,1,101.7166
307,2024-04-05,1,101.9053
308,2024-04-06,1,102.2974
309,2024-04-09,1,102.2204
